In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np

In [11]:
from fuel.datasets.cifar10 import CIFAR10

cifar_train = CIFAR10(("train",))
cifar_test  = CIFAR10(("test",))

In [3]:
from fuel.datasets.mnist import MNIST
from fuel.transformers import ScaleAndShift, Cast, Flatten
from fuel.streams import DataStream
from fuel.schemes import SequentialScheme, ShuffledScheme

MNIST.default_transformers = (
    (ScaleAndShift, [2.0 / 255.0, -1], {'which_sources': 'features'}),
    (Cast, [np.float32], {'which_sources': 'features'}), 
    (Flatten, [], {'which_sources': 'features'}) )

mnist_train = MNIST(("train",), subset=slice(None,50000))
#this stream will shuffle the MNIST set and return us batches of 100 examples
mnist_train_stream = DataStream.default_stream(
    mnist_train,
    iteration_scheme=ShuffledScheme(mnist_train.num_examples, 100))
                                               
mnist_validation = MNIST(("train",), subset=slice(50000, None))

# We will use larger portions for testing and validation
# as these dont do a backward pass and reauire less RAM.
mnist_validation_stream = DataStream.default_stream(
    mnist_validation, iteration_scheme=SequentialScheme(mnist_validation.num_examples, 250))
mnist_test = MNIST(("test",))
mnist_test_stream = DataStream.default_stream(
    mnist_test, iteration_scheme=SequentialScheme(mnist_test.num_examples, 250))

In [4]:
from modules.Network import Network
from modules.layers.SoftMaxLayer import SoftMaxLayer

In [5]:
net = Network([SoftMaxLayer(784, 10)])
net.train(mnist_train_stream, mnist_validation_stream, 10, 1e-2)

At batch #100, batch cost: 0.889680
At batch #200, batch cost: 0.643908
At batch #300, batch cost: 0.697066
At batch #400, batch cost: 0.493531
At batch #500, batch cost: 0.486165
After epoch 0: validation error: 11.000000%
At batch #100, batch cost: 0.485613
At batch #200, batch cost: 0.496772
At batch #300, batch cost: 0.398573
At batch #400, batch cost: 0.448305
At batch #500, batch cost: 0.368536
After epoch 1: validation error: 9.970000%
At batch #100, batch cost: 0.385483
At batch #200, batch cost: 0.212670
At batch #300, batch cost: 0.478623
At batch #400, batch cost: 0.393425
At batch #500, batch cost: 0.410299
After epoch 2: validation error: 9.370000%
At batch #100, batch cost: 0.522089
At batch #200, batch cost: 0.349896
At batch #300, batch cost: 0.366624
At batch #400, batch cost: 0.458458
At batch #500, batch cost: 0.340334
After epoch 3: validation error: 8.980000%
At batch #100, batch cost: 0.369078
At batch #200, batch cost: 0.316869
At batch #300, batch cost: 0.386168